#Regression workflow

# Import

In [ ]:
# --- pip installs ---
!pip install xgboost
!pip install --upgrade category_encoders
!pip install shap
!pip install catboost
!pip install lightgbm
!pip install mlxtend
!pip install lightgbm

In [ ]:
spark.read.format("parquet").load("/mnt/mainstorage/workspace/01_PROJETS/01_DSN_32_PredictionAbsenteisme/Production/base_duree.parquet")

In [ ]:
# IMPORTING LIBRARIES & MAIN PATH

import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import skew, norm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
%matplotlib inline

import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

#Ignore warnings
import warnings
warnings.filterwarnings(action="ignore")

# Defining the working data
# final_data = sqlContext.read.parquet("/mnt/mainstorage/workspace/01_PROJETS/01_DSN_32_PredictionAbsenteisme/Production/cout_freq_annuelle_tpt_distances_final_v2.parquet")
final_data = spark.read.format("parquet").load("/mnt/mainstorage/workspace/01_PROJETS/01_DSN_32_PredictionAbsenteisme/Production/base_duree.parquet")
data = final_data.toPandas()
#data['duree_moy'] = data['duree']/data['frequence']
drop_feats = ['SIRET', 'PER_ETUDE_DEB', 'CLE', 'DT_DEB_CTR_TRV', 'PER_ETUDE_FIN', 'CD_PTT_SAL', 'IDCC_GROUP2', 'LIBELLE_NATURE_CONTRAT', 'mois_', 'TRANCHE_AGE', 'Ind_enfant', 'LIBELLE_CSP_DataMAP', 'frequence']
data = data.drop(drop_feats, axis=1) #additional drops
target = 'duree'
data = data[(data[target] <= 31) & (data[target] > 0)]
data['SAL_PRECED'] = data['SAL_PRECED'].astype('float')
data['MORAL'] = data['MORAL'].astype('float')
#data = data[(data[target] > 0)]

X_train, X_val, y_train, y_val = train_test_split(data.drop(target, axis=1), data[target], test_size=0.25, random_state=123)

#Show the data
data.head()

In [ ]:
data.info()

# EDA

In [ ]:
data.describe()

# Feature correlation

## Heatmap

In [ ]:
f, ax = plt.subplots(figsize=(30, 25))
mat = data.corr('pearson')
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(mat, cmap=cmap, vmax=1, center=0, annot = True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
print(np.abs(mat[target]).sort_values(ascending=False))
plt.show()

## Numerical features

In [ ]:
for feature in data.select_dtypes(exclude='object').columns:
  pearson = float(mat.loc[target, feature])
  figure, ax = plt.subplots(1,2, figsize = (18,6), gridspec_kw={'width_ratios': [2, 1]})
  sns.regplot(data=data.iloc[:1000], x = feature, y=target, scatter_kws={'alpha':0.5}, ax = ax[0], order=1)
  ax[0].set_title(feature,fontweight="bold", size=14)
  ax[0].legend(['$Pearson=$ {:.2f}'.format(pearson)], loc = 'best')
  sns.histplot(data=data.iloc[:1000], x = feature, ax = ax[1], stat='probability')
  ax[1].tick_params(labelrotation=90)
  plt.show()
#sns.catplot(data=data.iloc[:1000], x = feature, y=target, kind="boxen")

## Categorical features

In [ ]:
max_distinct_values = 10
additional_cat_feats = [col for col in data.select_dtypes(exclude='object').columns if len(data.select_dtypes(exclude='object')[col].unique()) <= max_distinct_values]
cat_feats = [col for col in data.select_dtypes(include='object').columns if len(data.select_dtypes(include='object')[col].unique()) <= max_distinct_values]
for feature in cat_feats + additional_cat_feats:
  figure, ax = plt.subplots(1,3, figsize = (20,8))
  sns.stripplot(data=data.iloc[:1000], x = feature, y=target, ax = ax[0], alpha=0.5)
  ax[0].tick_params(labelrotation=90)
  sns.violinplot(data=data.iloc[:1000], x = feature, y=target, ax = ax[1])
  ax[1].tick_params(labelrotation=90)
  ax[1].set_title(feature,fontweight="bold", size=14)
  sns.boxplot(data=data.iloc[:1000], x = feature, y=target, ax = ax[2])
  ax[2].tick_params(labelrotation=90)
  plt.show()

## Feature interaction

In [ ]:
categorical_feature = "LIBELLE_CSP"
numerical_feature = "AGE"
plt.figure(figsize=(12,8))
sns.lmplot(x=numerical_feature, y=target, hue=categorical_feature, data=data.iloc[:1000])
plt.show()

In [ ]:
categorical_feature = "LIBELLE_CSP"
numerical_feature = "AGE"
sns.lmplot(
    x=numerical_feature, y=target, hue=categorical_feature, col=categorical_feature,
    data=data.iloc[:1000], scatter_kws={"edgecolor": 'w'}, col_wrap=3, height=4,
)

#  Data cleaning

Now that we have some insights about data, we need to preprocess them for the modeling part. The main steps are:

- Looking at potential NaN
- Dealing with outliers
- Dealing with categorical features (e.g. Dummy coding)
- Normalization of numerical features

N.B:

Usually, in a real-world project, the test data are not available until the end. For this reason, test data should contain the same type of data of the training set to preprocess them in the same way. Here, the test set is available. It contains some observations not present in the training dataset and,the use of dummy coding could raise several issues (I spent a lot of time figuring out why I was not able to make predictions on the test set). The easiest way to solve this problem (that is not applicable if test data are not available) is to concatenate Train and Test sets, preprocess, and divide them again.

## Data Score

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

def score_dataset_cv(X, y, model=XGBRegressor(), cv=5):
    # Label encoding for categoricals
    # Nan = -1
    for colname in X.select_dtypes(["category", "object"]): #there should be no categorical features at the end
        X[colname], _ = X[colname].factorize()
    #Scoring
    if cv <= 1:
      X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)
      model.fit(X_train, y_train)
      pred = model.predict(X_val)
      score = mean_squared_error(y_val, pred, squared=False)
      return score
    else:
      score = cross_val_score(
          model, X, y, cv=cv, scoring="neg_root_mean_squared_error", #metric can be changed
      )
      score = -1 * score.mean()
      return score
    
def score_dataset(X_train, X_val, y_train, y_val, model=XGBRegressor()):
    #Scoring
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    score = mean_squared_error(y_val, pred, squared=False)
    return score

In [ ]:
score_dataset_cv(data.iloc[:5000].drop(target, axis=1), data.iloc[:5000][target], cv=1)

## Dealing with NaN

3 approches :

- Drop Columns with Missing Values
- Imputation
- An Extension To Imputation

In [ ]:
# Looking at NaN % within the data
threshold = 20
nan = pd.DataFrame(data.isna().sum(), columns = ['NaN_sum'])
nan['Perc(%)'] = (nan['NaN_sum']/1460)*100
# nan = nan[nan['NaN_sum'] > 0]
nan = nan.sort_values(by = ['NaN_sum'])
nan['Usability'] = np.where(nan['Perc(%)'] > threshold, 'Discard', 'Keep')
nan

In [ ]:
# Plotting Nan
plt.figure(figsize = (15,5))
sns.barplot(x = nan.index, y = nan['Perc(%)'])
plt.xticks(rotation=45)
plt.title('Features containing Nan')
plt.xlabel('Features')
plt.ylabel('% of Missing Data')
plt.show()

Are we sure that all these nans are real missing values? Looking at the given description file, we can see how the majority of these nans reflect the absence of something, and for this reason, they are not nans. We can impute them (for numerical features) or substitute them with data in the file:

In [ ]:
# Converting non-numeric predictors stored as numbers into string

train_test['MSSubClass'] = train_test['MSSubClass'].apply(str)
train_test['YrSold'] = train_test['YrSold'].apply(str)
train_test['MoSold'] = train_test['MoSold'].apply(str)

# Filling Categorical NaN (That we know how to fill due to the description file )

train_test['Functional'] = train_test['Functional'].fillna('Typ')
train_test['Electrical'] = train_test['Electrical'].fillna("SBrkr")
train_test['KitchenQual'] = train_test['KitchenQual'].fillna("TA")
train_test['Exterior1st'] = train_test['Exterior1st'].fillna(train_test['Exterior1st'].mode()[0])
train_test['Exterior2nd'] = train_test['Exterior2nd'].fillna(train_test['Exterior2nd'].mode()[0])
train_test['SaleType'] = train_test['SaleType'].fillna(train_test['SaleType'].mode()[0])
train_test["PoolQC"] = train_test["PoolQC"].fillna("None")
train_test["Alley"] = train_test["Alley"].fillna("None")
train_test['FireplaceQu'] = train_test['FireplaceQu'].fillna("None")
train_test['Fence'] = train_test['Fence'].fillna("None")
train_test['MiscFeature'] = train_test['MiscFeature'].fillna("None")

for col in ('GarageArea', 'GarageCars'):
    train_test[col] = train_test[col].fillna(0)
        
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_test[col] = train_test[col].fillna('None')
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    train_test[col] = train_test[col].fillna('None')
    
    # Checking the features with NaN remained out

for col in train_test:
    if train_test[col].isna().sum() > 0:
        print(train_test[col][0])

In [ ]:
# Removing the useless variables

useless = ['GarageYrBlt','YearRemodAdd'] 
train_test = train_test.drop(useless, axis = 1)

# Imputing with KnnRegressor (we can also use different Imputers)

def impute_knn(df):
    ttn = train_test.select_dtypes(include=[np.number])
    ttc = train_test.select_dtypes(exclude=[np.number])

    cols_nan = ttn.columns[ttn.isna().any()].tolist()         # columns w/ nan 
    cols_no_nan = ttn.columns.difference(cols_nan).values     # columns w/n nan

    for col in cols_nan:
        imp_test = ttn[ttn[col].isna()]   # indicies which have missing data will become our test set
        imp_train = ttn.dropna()          # all indicies which which have no missing data 
        model = KNeighborsRegressor(n_neighbors=5)  # KNR Unsupervised Approach
        knr = model.fit(imp_train[cols_no_nan], imp_train[col])
        ttn.loc[ttn[col].isna(), col] = knr.predict(imp_test[cols_no_nan])
    
    return pd.concat([ttn,ttc],axis=1)

train_test = impute_knn(train_test)


objects = []
for i in train_test.columns:
    if train_test[i].dtype == object:
        objects.append(i)
train_test.update(train_test[objects].fillna('None'))

# # Checking NaN presence

for col in train_test:
    if train_test[col].isna().sum() > 0:
        print(train_test[col][0])

## Outliers

In [ ]:
X_train['SAL_PCT'] = np.where(X_train['SAL_PCT'] > 1, 1, X_train['SAL_PCT'])
X_val['SAL_PCT'] = np.where(X_val['SAL_PCT'] > 1, 1, X_val['SAL_PCT'])

# Feature Engineering

In [ ]:
data.head()

## Categorical features

3 approches :

- Drop Categorical Variables
- Ordinal Encoding
- One-Hot Encoding
- Target Encoding

###Commun

In [ ]:
X_train_cat = X_train.select_dtypes(include='object')
X_val_cat = X_val.select_dtypes(include='object')
X_train_num = X_train.select_dtypes(exclude='object')
X_val_num = X_val.select_dtypes(exclude='object')

In [ ]:
X_train_cat.head()

In [ ]:
cat_cols = data.select_dtypes(include='object').columns
cat_distinct_values = [len(data[col].unique()) for col in cat_cols]
pd.DataFrame(cat_distinct_values, index=cat_cols)

### One Hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
max_distinct_values = 30
onehot_cat_feats = [col for col in data.select_dtypes(include='object').columns if len(data.select_dtypes(include='object')[col].unique()) <= max_distinct_values]
encoder =  OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder.fit(X_train_cat[onehot_cat_feats])
cols = encoder.get_feature_names(onehot_cat_feats)
X_train_cat_onehot = pd.DataFrame(encoder.transform(X_train_cat[onehot_cat_feats]), columns = cols, index=X_train_cat.index)
X_val_cat_onehot = pd.DataFrame(encoder.transform(X_val_cat[onehot_cat_feats]), columns = cols, index=X_val_cat.index)


In [ ]:
X_train_cat_treated = X_train_cat_onehot
X_val_cat_treated = X_val_cat_onehot

### Ordinal Encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
cols = ['LIBELLE_CSP']

# Make copy to avoid changing original data 
X_train_cat_treated = X_train_cat[cols].copy()
X_val_cat_treated = X_val_cat[cols].copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
X_train_cat_treated[cols] = ordinal_encoder.fit_transform(X_train_cat[cols])
X_val_cat_treated[cols] = ordinal_encoder.transform(X_val_cat[cols])

### Target Encoding

In [ ]:
# Create target encoding
from category_encoders import MEstimateEncoder
cols = ['IDCC_GROUP2']

# Create the encoder instance. Choose m to control noise.
encoder = MEstimateEncoder(cols=cols, m=5.0)
# Fit the encoder on the encoding split. /!\ WHICH IS DIFFERENT THAN X_train !!!
encoder.fit(X_val_cat, y_val)
# Encode the Zipcode column to create the final training data
X_train_cat_treated[cols] = encoder.transform(X_train_cat)[cols]
X_val_cat_treated[cols] = encoder.transform(X_val_cat)[cols]

In [ ]:
X_train_cat_treated = pd.concat([X_train_cat_treated, X_train_cat_onehot], axis=1)
X_val_cat_treated = pd.concat([X_val_cat_treated, X_val_cat_onehot], axis=1)

In [ ]:
X_train_cat_treated.head()

## Numerical Features

### Normalize

In literature, acceptable values for skewness are between -0.5 and 0.5 while -2 and 2 for Kurtosis.

In [ ]:
def transform_data(series, function=np.log1p):
  """
  In literature, acceptable values for skewness are between -0.5 and 0.5 while -2 and 2 for Kurtosis.
  """ 
  from scipy import stats
  # Before transformation
  (mu, sigma) = norm.fit(series)
  fig, ax = plt.subplots(1,2, figsize= (15,5))
  fig.suptitle(f" qq-plot & distribution {series.name}", fontsize= 15)
  sm.qqplot(series, stats.t, distargs=(4,),fit=True, line="45", ax = ax[0])
  sns.distplot(series, kde = True, hist=True, fit = norm, ax = ax[1])
  ax[1].legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
              loc='best')
  plt.show()

  # After transformation
  series_log = np.log1p(series)
  (mu, sigma) = norm.fit(series_log)
  fig, ax = plt.subplots(1,2, figsize= (15,5))
  fig.suptitle(f" qq-plot & distribution AFTER transformation {series.name}", fontsize= 15)
  sm.qqplot(series_log, stats.t, distargs=(4,),fit=True, line="45", ax = ax[0])
  sns.distplot(series_log, kde = True, hist=True, fit = norm, ax = ax[1])
  ax[1].legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
              loc='best')
  plt.show()

  # Skew and kurt
  shap_t,shap_p = stats.shapiro(series)
  print("Skewness: %f" % abs(series).skew())
  print("Kurtosis: %f" % abs(series).kurt())
  print("Shapiro_Test: %f" % shap_t)
  print("Shapiro_Test: %f" % shap_p)

In [ ]:
transform_data(data[target], function=np.log1p)

Automatic un-skew process

In [ ]:
# Fetch all numeric features
numeric_features = train_test_dummy.dtypes[train_test_dummy.dtypes != object].index
skewed_features = train_test_dummy[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_features[skewed_features > 0.5]
skew_index = high_skew.index

# Normalize skewed features using log_transformation
    
for i in skew_index:
    train_test_dummy[i] = np.log1p(train_test_dummy[i])

### Scale

In [ ]:
X_train_num.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_num_treated = pd.DataFrame(scaler.fit_transform(X_train_num), columns= X_train_num.columns, index=X_train_num.index)
X_val_num_treated = pd.DataFrame(scaler.transform(X_val_num), columns= X_train_num.columns, index=X_val_num.index)

## PCA

PCA to create features and detect outliers

In [ ]:
from sklearn.decomposition import PCA

# Create principal components
pca = PCA()
X_train_pca = pca.fit_transform(X_train_num_scaled)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(X_train_pca.shape[1])]
X_train_pca = pd.DataFrame(X_train_pca, columns=component_names, index=X_train_num_scaled.index)

X_train_pca.head()

In [ ]:
def apply_pca(X, standardize=True):
    # Standardize
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Create principal components
    pca = PCA()
    X_pca = pca.fit_transform(X)
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names, index=X.index)
    # Create loadings
    loadings = pd.DataFrame(
        pca.components_.T,  # transpose the matrix of loadings
        columns=component_names,  # so the columns are the principal components
        index=X.columns,  # and the rows are the original features
    )
    return pca, X_pca, loadings


def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

In [ ]:
pca, X_pca, loadings = apply_pca(X_train_num, standardize=True)

In [ ]:
f, ax = plt.subplots(figsize=(20, 18))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(loadings, cmap=cmap, vmax=1, center=0, annot = True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title("PCA feature coefs ie. Loadings", fontsize=22)
plt.show()

In [ ]:
plot_variance(pca)

## Additional features

Ideas for additional features :
- Clustering for features with complicated relationship (latitude, longtitude for example)
- Product or sum of features including binary/categorical ones
- Target Encoding (train the encoder on a different dataset than the one being fitted by the model)

Make sure to rescaled them after

### Commun

In [ ]:
X_train_treated = pd.concat([X_train_num, X_train_cat_treated], axis=1)
X_val_treated = pd.concat([X_val_num, X_val_cat_treated], axis=1)

### Clustering

In [ ]:
X_train_treated.head()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Create cluster feature
kmeans = KMeans(n_clusters=6)
X_train_treated["REGION"] = kmeans.fit_predict(X_train_treated[['LATITUDE_ETB', 'LONGITUDE_ETB']])
X_train_treated["REGION"] = X_train_treated["REGION"].astype("object")
X_val_treated["REGION"] = kmeans.predict(X_val_treated[['LATITUDE_ETB', 'LONGITUDE_ETB']])
X_val_treated["REGION"] = X_val_treated["REGION"].astype("object")
X_train_treated.head()

In [ ]:
sns.relplot(
    x="LONGITUDE_ETB", y="LATITUDE_ETB", hue="REGION", data=X_val_treated.iloc[:1000], height=6,
)

In [ ]:
onehot_cat_feats = ['REGION']
encoder =  OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder.fit(X_train_treated[onehot_cat_feats])
cols = encoder.get_feature_names(onehot_cat_feats)
X_train_treated_onehot = pd.DataFrame(encoder.transform(X_train_treated[onehot_cat_feats]), columns = cols, index=X_train_treated.index)
X_val_treated_onehot = pd.DataFrame(encoder.transform(X_val_treated[onehot_cat_feats]), columns = cols, index=X_val_treated.index)
X_train_treated = pd.concat([X_train_treated, X_train_treated_onehot], axis=1)
X_val_treated = pd.concat([X_val_treated, X_val_treated_onehot], axis=1)
X_train_treated.drop(onehot_cat_feats, axis=1)
X_val_treated.drop(onehot_cat_feats, axis=1)

### Created features

In [ ]:
X_train_treated.head()

In [ ]:
# Create new features
train_test["SqFtPerRoom"] = train_test["GrLivArea"] / (train_test["TotRmsAbvGrd"] +
                                                       train_test["FullBath"] +
                                                       train_test["HalfBath"] +
                                                       train_test["KitchenAbvGr"])

train_test['Total_Home_Quality'] = train_test['OverallQual'] + train_test['OverallCond']

train_test['Total_Bathrooms'] = (train_test['FullBath'] + (0.5 * train_test['HalfBath']) +
                               train_test['BsmtFullBath'] + (0.5 * train_test['BsmtHalfBath']))

train_test["HighQualSF"] = train_test["GrLivArea"]+train_test["1stFlrSF"] + train_test["2ndFlrSF"]+0.5*train_test["GarageArea"]+0.5*train_test["TotalBsmtSF"]+1*train_test["MasVnrArea"]

# Training and Testing

## Pipeline

In [ ]:
import shap
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import Pool
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.ensemble import VotingRegressor

### Model Score

In [ ]:
import re
X_train_treated = X_train_treated.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_val_treated = X_val_treated.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
def score_model(model=LGBMRegressor(), scoring = mean_absolute_error):
    #Scoring
    model.fit(X_train_treated, y_train)
    pred = model.predict(X_val_treated)
    score = scoring(y_val, pred)
    return score

In [ ]:
(792799 * 6.13344834965373 + 337025 * 71.00711044385457) / (792799 + 337025)

MAE avec clustering des régions : 43.12486320047404
MAE sans clustering des régions : 43.111405813071954

In [ ]:
score_model()

In [ ]:
y_val.std()

In [ ]:
# Creation of the RMSE metric:
    
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, cv=5):
    rmse = np.sqrt(-cross_val_score(model, train, target_log, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

### Models

In [ ]:
# # 10 Fold Cross validation

# kf = KFold(n_splits=5, random_state=42, shuffle=True)

# cv_scores = []
# cv_std = []

# baseline_models = ['Linear_Reg.','Bayesian_Ridge_Reg.','LGBM_Reg.','SVR',
#                    'Dec_Tree_Reg.','Random_Forest_Reg.', 'XGB_Reg.',
#                    'Grad_Boost_Reg.','Cat_Boost_Reg.','Stacked_Reg.']

# # Linear Regression

# lreg = LinearRegression()
# score_lreg = cv_rmse(lreg)
# cv_scores.append(score_lreg.mean())
# cv_std.append(score_lreg.std())

# # Bayesian Ridge Regression

# brr = BayesianRidge(compute_score=True)
# score_brr = cv_rmse(brr)
# cv_scores.append(score_brr.mean())
# cv_std.append(score_brr.std())

# # Light Gradient Boost Regressor

# l_gbm = LGBMRegressor(objective='regression')
# score_l_gbm = cv_rmse(l_gbm)
# cv_scores.append(score_l_gbm.mean())
# cv_std.append(score_l_gbm.std())

# # Support Vector Regression

# svr = SVR()
# score_svr = cv_rmse(svr)
# cv_scores.append(score_svr.mean())
# cv_std.append(score_svr.std())

# # Decision Tree Regressor

# dtr = DecisionTreeRegressor()
# score_dtr = cv_rmse(dtr)
# cv_scores.append(score_dtr.mean())
# cv_std.append(score_dtr.std())

# # Random Forest Regressor

# rfr = RandomForestRegressor()
# score_rfr = cv_rmse(rfr)
# cv_scores.append(score_rfr.mean())
# cv_std.append(score_rfr.std())

# # XGB Regressor

# xgb = xgb.XGBRegressor()
# score_xgb = cv_rmse(xgb)
# cv_scores.append(score_xgb.mean())
# cv_std.append(score_xgb.std())

# # Gradient Boost Regressor

# gbr = GradientBoostingRegressor()
# score_gbr = cv_rmse(gbr)
# cv_scores.append(score_gbr.mean())
# cv_std.append(score_gbr.std())

# # Cat Boost Regressor

# catb = CatBoostRegressor()
# score_catb = cv_rmse(catb)
# cv_scores.append(score_catb.mean())
# cv_std.append(score_catb.std())

# # Stacked Regressor

# stack_gen = StackingRegressor(regressors=(CatBoostRegressor(),
#                                           LinearRegression(),
#                                           BayesianRidge(),
#                                           GradientBoostingRegressor()),
#                               meta_regressor = CatBoostRegressor(),
#                               use_features_in_secondary = True)

# score_stack_gen = cv_rmse(stack_gen)
# cv_scores.append(score_stack_gen.mean())
# cv_std.append(score_stack_gen.std())

# # vote_gen = VotingRegressor(estimators=[('Cat_Boost_Reg', CatBoostRegressor()), 
# #                                              ('Linear_Reg', LinearRegression()), 
# #                                              ('Bayesian_Ridge_Reg', BayesianRidge()),
# #                                              ('Grad_Boost_Reg', GradientBoostingRegressor()), 
# #                                              ('LGBM_Reg', LGBMRegressor(objective='regression'))
# #                                             ])

# # score_vote_gen = cv_rmse(vote_gen)
# # cv_scores.append(score_vote_gen.mean())
# # cv_std.append(score_vote_gen.std())

# final_cv_score = pd.DataFrame(baseline_models, columns = ['Regressors'])
# final_cv_score['RMSE_mean'] = cv_scores
# final_cv_score['RMSE_std'] = cv_std

In [ ]:
# final_cv_score

In [ ]:
# plt.figure(figsize = (12,8))
# sns.barplot(final_cv_score['Regressors'],final_cv_score['RMSE_mean'])
# plt.xlabel('Regressors', fontsize = 12)
# plt.ylabel('CV_Mean_RMSE', fontsize = 12)
# plt.xticks(rotation=45)
# plt.show()

### Shap

#### Global analysis

In [ ]:
# Best Model

cat = CatBoostRegressor()
cat_model = cat.fit(X_train_treated, y_train,
                     eval_set = (X_val_treated, y_val),
                     plot=True,
                     verbose = 0)
# cat = LGBMRegressor()
# cat_model = cat.fit(X_train_treated, y_train)

In [ ]:
cat_pred = cat_model.predict(X_val_treated)
cat_score = rmse(y_val, cat_pred)
cat_score

Now let's take a look at the top 20 most important variables for our model. This could give us further insight into the functioning of the algorithm and how and which data it uses most to arrive at the final prediction.

In [ ]:
# Features' importance of our model

feat_imp = cat_model.get_feature_importance(prettified=True)
feat_imp

In [ ]:
plt.figure(figsize = (12,8))
sns.barplot(feat_imp['Importances'][:40],feat_imp['Feature Id'][:40], orient = 'h')
plt.show()

In [ ]:
#Feature importance Interactive Plot 

train_pool = Pool(X_train_treated)
val_pool = Pool(X_val_treated)

explainer = shap.TreeExplainer(cat_model) # insert your model
shap_values = explainer.shap_values(train_pool) # insert your train Pool object

shap.initjs()
temp = shap.force_plot(explainer.expected_value, shap_values[:200,:], X_train_treated.iloc[:200,:], show = False)
# The plot represents just a slice of the Training data (200 observations)

In [ ]:
!pip install pickle

In [ ]:
import pickle
PATH = "/dbfs/FileStore/shapvalues"
filehandler = open(PATH, 'wb') 
pickle.dump(shap_values, filehandler)

In [ ]:
shap.summary_plot(shap_values, X_train_treated)

The above diagram represents each observation (x-axis) for the feature presented (y-axis). The x location of each dot on the x-axis reflects the impact of that feature on the model's predictions, while the color of the dot represents the value of that feature for that exact observation. Dots that pile up on the line show density.

#### Feature interaction

N.B: Catboost comes with a great method: ***get_feature_importance***. This method can be used to find important interactions among features. This is a huge advantage because it can give us insights about possible new features to create that can improve the performance.

In [ ]:
# Features' Interactions

train_data = Pool(X_train_treated)

interaction = cat_model.get_feature_importance(train_data, type="Interaction")
column_names = X_train_treated.columns.values 
interaction = pd.DataFrame(interaction, columns=["feature1", "feature2", "importance"])
interaction.feature1 = interaction.feature1.apply(lambda l: column_names[int(l)])
interaction.feature2 = interaction.feature2.apply(lambda l: column_names[int(l)])
interaction.head(20)

#### Error analysis

In [ ]:
def plot_shap_single(n_max = 200, n_indices = 10, sort = "worst"):
  """
  sort : worst, best, random
  """
  # Fits the explainer
  explainer = shap.Explainer(cat_model.predict, X_val_treated.iloc[:n_max])
  # Calculates the SHAP values - It takes some time
  shap_values_single = explainer(X_val_treated.iloc[:n_max])

  pred = cat_model.predict(X_val_treated.iloc[:n_max])
  truth = y_val.iloc[:n_max].to_numpy()
  rmse = np.abs(pred - truth)
  if sort == "worst":
    indices = np.flip(np.argsort(mse, axis=- 1, kind=None, order=None))
  elif sort == "best":
    indices = np.argsort(mse, axis=- 1, kind=None, order=None)
  else:
    indices = np.random.randint(n_max, size=n_indices)
  for i in range(n_indices):
    j = indices[i]
    print(f"real = {y_val.to_numpy()[j]} \npred = {cat_model.predict(X_val_treated.iloc[j])}")
    shap.plots.waterfall(shap_values_single[j])

In [ ]:
plot_shap_single()

## Hyperparameter tuning

Which are the default parameters used by CaboostRegressor? This is our real baseline, now we need to optimize the hyperparameters trying to tune the model to obtain a better performance.

In [ ]:
# Best model default paramters
cat_model.get_all_params()

In [ ]:
# # Preforming a Random Grid Search to find the best combination of parameters

# grid = {'iterations': [16000,20000,25000,30000],
#         'learning_rate': [0.04,0.05,0.01,0.002,0.005],
#         'depth': [1,2,3,4,5,6,7,8],
#         'l2_leaf_reg': [1, 3, 5, 9,13,15,17],
#         'max_leaves' : [8,10,12,14,16,32,64],
#         'early_stopping_rounds': [200],
#         'model_size_reg' : [0.2,0.5,0.7,0.9]}

# final_model = CatBoostRegressor()
# randomized_search_result = final_model.randomized_search(grid,
#                                                    X = X_train,
#                                                    y= y_train,
#                                                    verbose = False,
#                                                    plot=True)
                                                   

In [ ]:
# randomized_search_result['params']

In [ ]:
# Final Cat-Boost Regressor

params = {'max_leaves': 8,
          'depth': 3,
          'od_wait': 200,
          'l2_leaf_reg': 3,
          'iterations': 200000,
          'model_size_reg': 0.7,
          'learning_rate': 0.05,
          'random_seed': 42 }

# params = {'iterations': 4000,
#           'learning_rate': 0.002,
#           'depth': 4,
#           'l2_leaf_reg': 1,
#           'eval_metric':'RMSE',
#           'max_leaves':16,
#           'early_stopping_rounds': 200,
#           'model_size_reg':0.7,
#           'verbose': 200,
#           'random_seed': 42}
         
cat_f = CatBoostRegressor(**params)
cat_model_f = cat_f.fit(X_train,y_train,
                     eval_set = (X_val,y_val),
                     plot=True,
                     verbose = False)

catf_pred = cat_model_f.predict(X_val)
catf_score = rmse(y_val, catf_pred)

In [ ]:
catf_score

# Submission

In [ ]:
# Test CSV Submission

test_pred = cat_model_f.predict(test)
submission = pd.DataFrame(test_id, columns = ['Id'])
test_pred = np.expm1(test_pred)
submission['SalePrice'] = test_pred 
submission.head()

In [ ]:
# Saving the results in a csv file

submission.to_csv("submission.csv", index = False, header = True)